In [351]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.svm import SVR


from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
from sklearn.ensemble import VotingRegressor,RandomForestRegressor,StackingRegressor

from sklearn.metrics import mean_squared_error
from sklearn.decomposition import TruncatedSVD

import warnings
warnings.filterwarnings('ignore')

### Load dataset 

In [189]:
df = pd.read_csv('../week1/data/train.csv')

### Split train, valid 

In [190]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

### Now use your train_df for training and test_df for evaluation.
Note: do not use test_df for training

In [191]:
train_df

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
254,255,20,RL,70.0,8400,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,6,2010,WD,Normal,145000
1066,1067,60,RL,59.0,7837,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2009,WD,Normal,178000
638,639,30,RL,67.0,8777,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,5,2008,WD,Normal,85000
799,800,50,RL,60.0,7200,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal,175000
380,381,50,RL,50.0,5000,Pave,Pave,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,127000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1095,1096,20,RL,78.0,9317,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,3,2007,WD,Normal,176432
1130,1131,50,RL,65.0,7804,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,12,2009,WD,Normal,135000
1294,1295,20,RL,60.0,8172,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2006,WD,Normal,115000
860,861,50,RL,55.0,7642,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,NaN,0,6,2007,WD,Normal,189950


In [192]:
X_train, y_train = train_df.drop(columns='SalePrice'),train_df['SalePrice']
X_test, y_test = test_df.drop(columns='SalePrice'),test_df['SalePrice']

In [193]:
X_train.shape

(1168, 80)

In [194]:
X_test.shape

(292, 80)

In [195]:
y_train

254     145000
1066    178000
638      85000
799     175000
380     127000
         ...  
1095    176432
1130    135000
1294    115000
860     189950
1126    174000
Name: SalePrice, Length: 1168, dtype: int64

In [196]:
drop_features = []
def preprocess():
    #If feature missing more than 60% -> drop
    miss_feature = []
    for i in X_train:
        if(X_train.shape[0] - X_train[i].isna().sum() != 0):
            miss_feature.append(i)
    df_new_train = X_train
    for i in miss_feature:
        if(X_train[i].isna().sum()/X_train[i].shape[0] >= 0.6):
            df_new_train = df_new_train.drop([i],axis=1)
            drop_features.append(i)
            print(i)
            print(X_train[i].isna().sum()/X_train[i].shape[0])

    #drop feature id, because haven't much information 
    drop_features.append('Id')
    df_new_train = df_new_train.drop(['Id'],axis=1) 
    
    #test
    df_new_test = X_test
    for i in drop_features:
        df_new_test = df_new_test.drop([i],axis=1)
    
        
    # fill missing values = mean or mode
    for i in df_new_train:
        if(df_new_train[i].dtype == 'object'):
            df_new_train[i] = df_new_train[i].fillna(df_new_train[i].mode()[0])
            df_new_test[i] = df_new_test[i].fillna(df_new_train[i].mode()[0]) # test
        else :
            df_new_train[i] = df_new_train[i].fillna(df_new_train[i].mean())
            df_new_test[i] = df_new_test[i].fillna(df_new_train[i].mean()) #test
            
    return df_new_train,df_new_test

In [197]:
df_new_train,df_new_test = preprocess()

Alley
0.9366438356164384
PoolQC
0.9948630136986302
Fence
0.800513698630137
MiscFeature
0.9606164383561644


In [198]:
print(df_new_test.shape)
print(df_new_train.shape)

(292, 75)
(1168, 75)


In [283]:
def encoder(scaler = 0,degree = 0):
    #init
    Ordinal_Encoder = ['LotShape', 'LandSlope','ExterQual','ExterCond','BsmtQual','BsmtCond','BsmtExposure','HeatingQC',
                  'KitchenQual','FireplaceQu','GarageQual','GarageCond']
    Onehot_Encoder = []
    number_feature = []
    for i in df_new_train:
        if(df_new_train[i].dtype == 'object' and i not in Ordinal_Encoder and i not in drop_features):
            Onehot_Encoder.append(i)
        if(df_new_train[i].dtype != 'object'):
            number_feature.append(i)

    # ordinal Encoder
    oe = OrdinalEncoder()
    OrdEncoder_train = oe.fit_transform(df_new_train[Ordinal_Encoder])
    OrdEncoder_test = oe.transform(df_new_test[Ordinal_Encoder])
    
    #one hot encoder
    ohe = OneHotEncoder(handle_unknown='ignore')
    OneEncoder_train = ohe.fit_transform(df_new_train[Onehot_Encoder]).toarray()
    OneEncoder_test = ohe.transform(df_new_test[Onehot_Encoder]).toarray()
    
    
    # concat
    train = np.concatenate((OrdEncoder_train,OneEncoder_train), axis=1)
    train = np.concatenate((train,np.array(df_new_train[number_feature])), axis=1)
    test = np.concatenate((OrdEncoder_test,OneEncoder_test), axis=1)
    test = np.concatenate((test,np.array(df_new_test[number_feature])), axis=1)
    
    #Standard Scaler
    if(scaler != 0):
        ss = StandardScaler()
        train = ss.fit_transform(train)
        test = ss.transform(test)
    
    #Polynomial Regression
    if(degree > 0):
        poly_reg = PolynomialFeatures(degree=degree)
        train = poly_reg.fit_transform(train)
        test = poly_reg.transform(test)
    
    #return test,train
    return train,test

In [337]:
def model(train, test):
    model1 = LinearRegression()
    model1.fit(train,y_train)
    print('LinearRegression: ',end='')
    print(mean_squared_error(y_test, model1.predict(test), squared=False))
    
    model2 = SVR()
    model2.fit(train,y_train)
    print('SVR: ',end='')
    print(mean_squared_error(y_test, model2.predict(test), squared=False))
    
    model3 = Ridge()
    model3.fit(train,y_train)
    print('Ridge: ',end='')
    print(mean_squared_error(y_test, model3.predict(test), squared=False))
    
    model4 = Lasso()
    model4.fit(train,y_train)
    print('Lasso: ',end='')
    print(mean_squared_error(y_test, model4.predict(test), squared=False))
    
    model6 = LGBMRegressor()
    model6.fit(train,y_train)
    print('LGBMRegressor: ',end='')
    print(mean_squared_error(y_test, model6.predict(test), squared=False))
    
    model7 = RandomForestRegressor()
    model7.fit(train,y_train)
    print('RandomForestRegressor: ',end='')
    print(mean_squared_error(y_test, model7.predict(test), squared=False))
    
    model5 = CatBoostRegressor()
    model5.fit(train,y_train)
    print('CatBoostRegressor: ',end='')
    print(mean_squared_error(y_test, model5.predict(test), squared=False))

In [338]:
train,test = encoder()
model(train,test)

LinearRegression: 1227003.9020006363
SVR: 88653.02105186372
Ridge: 30658.372542634355
Lasso: 29479.740762735906
LGBMRegressor: 29675.58539690644
RandomForestRegressor: 29004.33423271067
Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 4.25ms	remaining: 4.24s
1:	learn: 73575.3956012	total: 8.52ms	remaining: 4.25s
2:	learn: 71722.9144612	total: 12.8ms	remaining: 4.24s
3:	learn: 70033.8231191	total: 17.5ms	remaining: 4.35s
4:	learn: 68312.9975773	total: 21.7ms	remaining: 4.31s
5:	learn: 66800.0626965	total: 25.9ms	remaining: 4.28s
6:	learn: 65197.7255418	total: 30ms	remaining: 4.26s
7:	learn: 63706.4232338	total: 34.4ms	remaining: 4.26s
8:	learn: 62233.4227708	total: 38.5ms	remaining: 4.24s
9:	learn: 60819.7728303	total: 42.8ms	remaining: 4.24s
10:	learn: 59475.2948358	total: 49ms	remaining: 4.4s
11:	learn: 58301.3539842	total: 53.2ms	remaining: 4.38s
12:	learn: 57025.9811731	total: 59.1ms	remaining: 4.49s
13:	learn: 55853.2653197	total: 63.9ms	remaining: 4.5s
14:	learn: 54656

170:	learn: 18130.2004547	total: 875ms	remaining: 4.24s
171:	learn: 18107.6641165	total: 882ms	remaining: 4.25s
172:	learn: 18081.8818769	total: 890ms	remaining: 4.25s
173:	learn: 18035.1784031	total: 896ms	remaining: 4.25s
174:	learn: 17987.8209398	total: 901ms	remaining: 4.25s
175:	learn: 17961.2862258	total: 912ms	remaining: 4.27s
176:	learn: 17907.7687165	total: 916ms	remaining: 4.26s
177:	learn: 17869.6092229	total: 923ms	remaining: 4.26s
178:	learn: 17813.5374091	total: 928ms	remaining: 4.25s
179:	learn: 17761.9685919	total: 934ms	remaining: 4.25s
180:	learn: 17711.5766537	total: 939ms	remaining: 4.25s
181:	learn: 17689.3815877	total: 943ms	remaining: 4.24s
182:	learn: 17659.8585436	total: 948ms	remaining: 4.23s
183:	learn: 17645.8539648	total: 954ms	remaining: 4.23s
184:	learn: 17631.2646328	total: 958ms	remaining: 4.22s
185:	learn: 17609.5299147	total: 962ms	remaining: 4.21s
186:	learn: 17575.9852150	total: 967ms	remaining: 4.2s
187:	learn: 17532.4139252	total: 971ms	remaining:

337:	learn: 13097.0102101	total: 1.66s	remaining: 3.25s
338:	learn: 13073.0241130	total: 1.67s	remaining: 3.25s
339:	learn: 13060.9252991	total: 1.67s	remaining: 3.25s
340:	learn: 13045.6898300	total: 1.68s	remaining: 3.24s
341:	learn: 13013.8677645	total: 1.68s	remaining: 3.24s
342:	learn: 13009.7271926	total: 1.69s	remaining: 3.24s
343:	learn: 13006.8473988	total: 1.69s	remaining: 3.23s
344:	learn: 12986.3632598	total: 1.7s	remaining: 3.22s
345:	learn: 12966.3153513	total: 1.7s	remaining: 3.22s
346:	learn: 12942.0147857	total: 1.71s	remaining: 3.21s
347:	learn: 12918.5293914	total: 1.71s	remaining: 3.21s
348:	learn: 12892.2306530	total: 1.72s	remaining: 3.21s
349:	learn: 12870.2920245	total: 1.72s	remaining: 3.2s
350:	learn: 12837.1951353	total: 1.73s	remaining: 3.19s
351:	learn: 12816.4665972	total: 1.73s	remaining: 3.19s
352:	learn: 12796.5872819	total: 1.74s	remaining: 3.18s
353:	learn: 12786.0039150	total: 1.74s	remaining: 3.18s
354:	learn: 12757.6797500	total: 1.74s	remaining: 3

505:	learn: 10221.4125045	total: 2.44s	remaining: 2.38s
506:	learn: 10206.3029088	total: 2.45s	remaining: 2.38s
507:	learn: 10204.9697376	total: 2.46s	remaining: 2.38s
508:	learn: 10200.3818958	total: 2.46s	remaining: 2.37s
509:	learn: 10192.8764155	total: 2.47s	remaining: 2.37s
510:	learn: 10175.8673840	total: 2.47s	remaining: 2.37s
511:	learn: 10168.1883388	total: 2.48s	remaining: 2.36s
512:	learn: 10153.8799158	total: 2.48s	remaining: 2.36s
513:	learn: 10150.1571456	total: 2.49s	remaining: 2.35s
514:	learn: 10145.1967538	total: 2.49s	remaining: 2.35s
515:	learn: 10127.9456372	total: 2.5s	remaining: 2.35s
516:	learn: 10124.2336981	total: 2.5s	remaining: 2.34s
517:	learn: 10108.0595440	total: 2.51s	remaining: 2.33s
518:	learn: 10096.5632717	total: 2.51s	remaining: 2.33s
519:	learn: 10074.9803325	total: 2.52s	remaining: 2.33s
520:	learn: 10069.2108262	total: 2.52s	remaining: 2.32s
521:	learn: 10051.1007662	total: 2.53s	remaining: 2.31s
522:	learn: 10048.7969716	total: 2.53s	remaining: 

670:	learn: 8491.4867861	total: 3.43s	remaining: 1.68s
671:	learn: 8481.7454797	total: 3.44s	remaining: 1.68s
672:	learn: 8471.2407901	total: 3.45s	remaining: 1.68s
673:	learn: 8466.5793539	total: 3.46s	remaining: 1.67s
674:	learn: 8457.1890479	total: 3.47s	remaining: 1.67s
675:	learn: 8453.3252804	total: 3.47s	remaining: 1.66s
676:	learn: 8441.3827722	total: 3.48s	remaining: 1.66s
677:	learn: 8432.6723113	total: 3.49s	remaining: 1.66s
678:	learn: 8413.1784692	total: 3.5s	remaining: 1.65s
679:	learn: 8405.0547936	total: 3.5s	remaining: 1.65s
680:	learn: 8401.6567782	total: 3.51s	remaining: 1.65s
681:	learn: 8385.2898327	total: 3.52s	remaining: 1.64s
682:	learn: 8372.7835640	total: 3.53s	remaining: 1.64s
683:	learn: 8363.7749390	total: 3.54s	remaining: 1.63s
684:	learn: 8360.0675651	total: 3.54s	remaining: 1.63s
685:	learn: 8343.4746885	total: 3.55s	remaining: 1.62s
686:	learn: 8331.0209699	total: 3.56s	remaining: 1.62s
687:	learn: 8314.5723427	total: 3.56s	remaining: 1.62s
688:	learn: 

845:	learn: 6984.7847640	total: 4.63s	remaining: 844ms
846:	learn: 6975.3802861	total: 4.64s	remaining: 839ms
847:	learn: 6964.7933001	total: 4.65s	remaining: 834ms
848:	learn: 6957.7116955	total: 4.66s	remaining: 830ms
849:	learn: 6956.0002618	total: 4.68s	remaining: 825ms
850:	learn: 6949.7843910	total: 4.69s	remaining: 821ms
851:	learn: 6942.4049095	total: 4.7s	remaining: 816ms
852:	learn: 6931.5502123	total: 4.71s	remaining: 811ms
853:	learn: 6930.9262175	total: 4.72s	remaining: 806ms
854:	learn: 6927.1007430	total: 4.72s	remaining: 801ms
855:	learn: 6916.8993704	total: 4.74s	remaining: 797ms
856:	learn: 6908.9579416	total: 4.75s	remaining: 792ms
857:	learn: 6899.6300677	total: 4.75s	remaining: 787ms
858:	learn: 6895.4205795	total: 4.77s	remaining: 783ms
859:	learn: 6886.6816237	total: 4.78s	remaining: 779ms
860:	learn: 6883.7292261	total: 4.8s	remaining: 774ms
861:	learn: 6879.7757286	total: 4.81s	remaining: 771ms
862:	learn: 6877.3999820	total: 4.82s	remaining: 765ms
863:	learn: 

CatBoostRegressor: 25907.39504664792


In [339]:
train,test = encoder(degree=2)
model(train,test)

LinearRegression: 92622.11271654429
SVR: 88666.24089826437
Ridge: 45761.317053884086
Lasso: 43521.4130902992
LGBMRegressor: 28229.462890156698
RandomForestRegressor: 27422.91656838602
Learning rate set to 0.040124
0:	learn: 75169.4491500	total: 2.12s	remaining: 35m 13s
1:	learn: 73230.7701004	total: 2.42s	remaining: 20m 5s
2:	learn: 71516.3003091	total: 2.7s	remaining: 14m 57s
3:	learn: 69767.4410889	total: 3s	remaining: 12m 26s
4:	learn: 68062.1407375	total: 3.29s	remaining: 10m 55s
5:	learn: 66449.7321094	total: 3.57s	remaining: 9m 51s
6:	learn: 64821.6048109	total: 3.84s	remaining: 9m 4s
7:	learn: 63344.2543840	total: 4.11s	remaining: 8m 29s
8:	learn: 61923.9830693	total: 4.38s	remaining: 8m 2s
9:	learn: 60530.2971242	total: 4.65s	remaining: 7m 40s
10:	learn: 59217.4072432	total: 4.93s	remaining: 7m 23s
11:	learn: 57883.9926161	total: 5.21s	remaining: 7m 8s
12:	learn: 56652.8940553	total: 5.48s	remaining: 6m 56s
13:	learn: 55392.2630314	total: 5.75s	remaining: 6m 45s
14:	learn: 5428

143:	learn: 17944.4416538	total: 41.7s	remaining: 4m 8s
144:	learn: 17878.6697806	total: 42s	remaining: 4m 7s
145:	learn: 17809.0783225	total: 42.3s	remaining: 4m 7s
146:	learn: 17742.9196238	total: 42.5s	remaining: 4m 6s
147:	learn: 17687.1330760	total: 42.8s	remaining: 4m 6s
148:	learn: 17673.5151876	total: 43.1s	remaining: 4m 5s
149:	learn: 17645.7851282	total: 43.2s	remaining: 4m 4s
150:	learn: 17602.6534804	total: 43.5s	remaining: 4m 4s
151:	learn: 17561.2364299	total: 43.8s	remaining: 4m 4s
152:	learn: 17540.9774895	total: 44s	remaining: 4m 3s
153:	learn: 17465.9979966	total: 44.3s	remaining: 4m 3s
154:	learn: 17427.7580733	total: 44.5s	remaining: 4m 2s
155:	learn: 17387.8621159	total: 44.8s	remaining: 4m 2s
156:	learn: 17349.1730613	total: 45.1s	remaining: 4m 2s
157:	learn: 17329.3083520	total: 45.3s	remaining: 4m 1s
158:	learn: 17274.7978003	total: 45.6s	remaining: 4m 1s
159:	learn: 17210.1963230	total: 45.9s	remaining: 4m
160:	learn: 17154.9304767	total: 46.1s	remaining: 4m
16

287:	learn: 12626.0427317	total: 1m 20s	remaining: 3m 17s
288:	learn: 12600.2557304	total: 1m 20s	remaining: 3m 17s
289:	learn: 12572.9226912	total: 1m 20s	remaining: 3m 17s
290:	learn: 12532.7542263	total: 1m 20s	remaining: 3m 17s
291:	learn: 12523.7706082	total: 1m 21s	remaining: 3m 16s
292:	learn: 12486.2704994	total: 1m 21s	remaining: 3m 16s
293:	learn: 12448.7549723	total: 1m 21s	remaining: 3m 16s
294:	learn: 12420.1509385	total: 1m 21s	remaining: 3m 15s
295:	learn: 12411.3709379	total: 1m 22s	remaining: 3m 15s
296:	learn: 12373.3900182	total: 1m 22s	remaining: 3m 15s
297:	learn: 12365.2402321	total: 1m 22s	remaining: 3m 14s
298:	learn: 12356.8553608	total: 1m 22s	remaining: 3m 14s
299:	learn: 12325.4101848	total: 1m 23s	remaining: 3m 14s
300:	learn: 12296.8880005	total: 1m 23s	remaining: 3m 13s
301:	learn: 12269.5549325	total: 1m 23s	remaining: 3m 13s
302:	learn: 12255.0495336	total: 1m 24s	remaining: 3m 13s
303:	learn: 12247.0061227	total: 1m 24s	remaining: 3m 12s
304:	learn: 12

430:	learn: 9822.9622354	total: 1m 58s	remaining: 2m 36s
431:	learn: 9814.6404086	total: 1m 59s	remaining: 2m 36s
432:	learn: 9789.2140540	total: 1m 59s	remaining: 2m 36s
433:	learn: 9782.1432035	total: 1m 59s	remaining: 2m 35s
434:	learn: 9775.8492950	total: 1m 59s	remaining: 2m 35s
435:	learn: 9755.4056905	total: 2m	remaining: 2m 35s
436:	learn: 9747.7755235	total: 2m	remaining: 2m 35s
437:	learn: 9728.3458937	total: 2m	remaining: 2m 34s
438:	learn: 9707.9603159	total: 2m	remaining: 2m 34s
439:	learn: 9670.4662894	total: 2m 1s	remaining: 2m 34s
440:	learn: 9654.8168861	total: 2m 1s	remaining: 2m 34s
441:	learn: 9635.7495857	total: 2m 1s	remaining: 2m 33s
442:	learn: 9627.3260837	total: 2m 2s	remaining: 2m 33s
443:	learn: 9612.5747459	total: 2m 2s	remaining: 2m 33s
444:	learn: 9604.6246693	total: 2m 2s	remaining: 2m 33s
445:	learn: 9581.1157963	total: 2m 2s	remaining: 2m 32s
446:	learn: 9576.9899936	total: 2m 3s	remaining: 2m 32s
447:	learn: 9557.5286162	total: 2m 3s	remaining: 2m 32s

576:	learn: 7846.5662923	total: 2m 43s	remaining: 1m 59s
577:	learn: 7840.3797936	total: 2m 43s	remaining: 1m 59s
578:	learn: 7828.8286529	total: 2m 43s	remaining: 1m 58s
579:	learn: 7826.0930379	total: 2m 43s	remaining: 1m 58s
580:	learn: 7807.8984890	total: 2m 44s	remaining: 1m 58s
581:	learn: 7795.5589306	total: 2m 44s	remaining: 1m 58s
582:	learn: 7785.6510922	total: 2m 44s	remaining: 1m 57s
583:	learn: 7768.8723607	total: 2m 45s	remaining: 1m 57s
584:	learn: 7750.7401980	total: 2m 45s	remaining: 1m 57s
585:	learn: 7732.2549355	total: 2m 45s	remaining: 1m 57s
586:	learn: 7717.4920802	total: 2m 46s	remaining: 1m 56s
587:	learn: 7708.3216647	total: 2m 46s	remaining: 1m 56s
588:	learn: 7690.1502593	total: 2m 46s	remaining: 1m 56s
589:	learn: 7671.5808491	total: 2m 47s	remaining: 1m 56s
590:	learn: 7664.7587634	total: 2m 47s	remaining: 1m 55s
591:	learn: 7648.7400483	total: 2m 47s	remaining: 1m 55s
592:	learn: 7632.0284058	total: 2m 48s	remaining: 1m 55s
593:	learn: 7621.4986726	total:

721:	learn: 6572.6226637	total: 3m 29s	remaining: 1m 20s
722:	learn: 6569.9850567	total: 3m 30s	remaining: 1m 20s
723:	learn: 6568.0698359	total: 3m 30s	remaining: 1m 20s
724:	learn: 6566.2084367	total: 3m 30s	remaining: 1m 19s
725:	learn: 6556.4974405	total: 3m 31s	remaining: 1m 19s
726:	learn: 6549.0279432	total: 3m 31s	remaining: 1m 19s
727:	learn: 6546.5272149	total: 3m 31s	remaining: 1m 19s
728:	learn: 6538.9460164	total: 3m 32s	remaining: 1m 18s
729:	learn: 6528.5887656	total: 3m 32s	remaining: 1m 18s
730:	learn: 6513.4634698	total: 3m 33s	remaining: 1m 18s
731:	learn: 6506.9469347	total: 3m 33s	remaining: 1m 18s
732:	learn: 6492.5484758	total: 3m 33s	remaining: 1m 17s
733:	learn: 6484.2387672	total: 3m 34s	remaining: 1m 17s
734:	learn: 6478.1500840	total: 3m 34s	remaining: 1m 17s
735:	learn: 6471.8672611	total: 3m 34s	remaining: 1m 17s
736:	learn: 6459.2965863	total: 3m 35s	remaining: 1m 16s
737:	learn: 6457.4744035	total: 3m 35s	remaining: 1m 16s
738:	learn: 6446.1036994	total:

868:	learn: 5481.4527687	total: 4m 20s	remaining: 39.3s
869:	learn: 5477.6552585	total: 4m 21s	remaining: 39s
870:	learn: 5470.9174705	total: 4m 21s	remaining: 38.7s
871:	learn: 5464.3551027	total: 4m 21s	remaining: 38.4s
872:	learn: 5455.2957246	total: 4m 22s	remaining: 38.1s
873:	learn: 5450.8898662	total: 4m 22s	remaining: 37.8s
874:	learn: 5446.3288984	total: 4m 22s	remaining: 37.5s
875:	learn: 5430.3565651	total: 4m 22s	remaining: 37.2s
876:	learn: 5424.2680047	total: 4m 23s	remaining: 36.9s
877:	learn: 5422.5902149	total: 4m 23s	remaining: 36.6s
878:	learn: 5417.4622804	total: 4m 23s	remaining: 36.3s
879:	learn: 5413.6162782	total: 4m 24s	remaining: 36s
880:	learn: 5407.9298528	total: 4m 24s	remaining: 35.7s
881:	learn: 5398.4674153	total: 4m 24s	remaining: 35.4s
882:	learn: 5397.0973643	total: 4m 24s	remaining: 35.1s
883:	learn: 5391.0170557	total: 4m 25s	remaining: 34.8s
884:	learn: 5385.0883722	total: 4m 25s	remaining: 34.5s
885:	learn: 5376.6177970	total: 4m 25s	remaining: 34

In [340]:
train,test = encoder()
svd = TruncatedSVD(n_components=100, n_iter=7, random_state=42)
train = svd.fit_transform(train)
test = svd.transform(test)

In [341]:
model(train,test)

LinearRegression: 34495.80704210149
SVR: 88653.09406842804
Ridge: 34429.96680308513
Lasso: 34496.02588933019
LGBMRegressor: 35309.565235172246
RandomForestRegressor: 38910.4893339053
Learning rate set to 0.040124
0:	learn: 75601.2851080	total: 20.9ms	remaining: 20.8s
1:	learn: 74198.8879905	total: 38.9ms	remaining: 19.4s
2:	learn: 72689.9961200	total: 58.2ms	remaining: 19.3s
3:	learn: 71513.6712489	total: 78.3ms	remaining: 19.5s
4:	learn: 70247.1734695	total: 99.9ms	remaining: 19.9s
5:	learn: 68935.7271788	total: 120ms	remaining: 19.9s
6:	learn: 67703.0896364	total: 139ms	remaining: 19.7s
7:	learn: 66588.4424654	total: 157ms	remaining: 19.4s
8:	learn: 65374.4334750	total: 179ms	remaining: 19.7s
9:	learn: 64096.3133190	total: 199ms	remaining: 19.7s
10:	learn: 62917.4196864	total: 217ms	remaining: 19.5s
11:	learn: 61848.4013627	total: 237ms	remaining: 19.5s
12:	learn: 60674.5625164	total: 258ms	remaining: 19.6s
13:	learn: 59540.5638524	total: 278ms	remaining: 19.6s
14:	learn: 58477.40963

150:	learn: 21487.0405525	total: 3.3s	remaining: 18.6s
151:	learn: 21415.5678641	total: 3.32s	remaining: 18.5s
152:	learn: 21321.9502841	total: 3.35s	remaining: 18.5s
153:	learn: 21220.6603643	total: 3.37s	remaining: 18.5s
154:	learn: 21156.7226004	total: 3.39s	remaining: 18.5s
155:	learn: 21054.2165658	total: 3.42s	remaining: 18.5s
156:	learn: 20956.4940690	total: 3.44s	remaining: 18.5s
157:	learn: 20871.8136267	total: 3.46s	remaining: 18.5s
158:	learn: 20789.0410343	total: 3.49s	remaining: 18.5s
159:	learn: 20722.0465858	total: 3.52s	remaining: 18.5s
160:	learn: 20637.9984479	total: 3.54s	remaining: 18.4s
161:	learn: 20531.0519082	total: 3.56s	remaining: 18.4s
162:	learn: 20460.2406710	total: 3.58s	remaining: 18.4s
163:	learn: 20397.7867350	total: 3.59s	remaining: 18.3s
164:	learn: 20297.0086351	total: 3.61s	remaining: 18.3s
165:	learn: 20227.4695040	total: 3.63s	remaining: 18.3s
166:	learn: 20149.2587400	total: 3.66s	remaining: 18.2s
167:	learn: 20077.8102167	total: 3.68s	remaining:

298:	learn: 13138.8715084	total: 7.4s	remaining: 17.4s
299:	learn: 13108.8637249	total: 7.43s	remaining: 17.3s
300:	learn: 13062.5696265	total: 7.47s	remaining: 17.3s
301:	learn: 13030.5543799	total: 7.5s	remaining: 17.3s
302:	learn: 12983.1432359	total: 7.53s	remaining: 17.3s
303:	learn: 12947.7342764	total: 7.56s	remaining: 17.3s
304:	learn: 12919.6260815	total: 7.59s	remaining: 17.3s
305:	learn: 12876.3924488	total: 7.62s	remaining: 17.3s
306:	learn: 12856.7757840	total: 7.64s	remaining: 17.2s
307:	learn: 12841.2754239	total: 7.66s	remaining: 17.2s
308:	learn: 12791.3589817	total: 7.68s	remaining: 17.2s
309:	learn: 12760.7231294	total: 7.71s	remaining: 17.2s
310:	learn: 12733.2351669	total: 7.73s	remaining: 17.1s
311:	learn: 12693.2096088	total: 7.75s	remaining: 17.1s
312:	learn: 12676.8613722	total: 7.77s	remaining: 17.1s
313:	learn: 12647.5455229	total: 7.79s	remaining: 17s
314:	learn: 12614.6690518	total: 7.81s	remaining: 17s
315:	learn: 12601.6668244	total: 7.84s	remaining: 17s


450:	learn: 8907.3953606	total: 10.9s	remaining: 13.2s
451:	learn: 8901.8066824	total: 10.9s	remaining: 13.2s
452:	learn: 8877.2214043	total: 10.9s	remaining: 13.2s
453:	learn: 8856.4047333	total: 10.9s	remaining: 13.2s
454:	learn: 8823.8767929	total: 11s	remaining: 13.1s
455:	learn: 8807.2275351	total: 11s	remaining: 13.1s
456:	learn: 8792.7112654	total: 11s	remaining: 13.1s
457:	learn: 8764.2066425	total: 11s	remaining: 13.1s
458:	learn: 8749.6683512	total: 11.1s	remaining: 13s
459:	learn: 8725.7777759	total: 11.1s	remaining: 13s
460:	learn: 8705.7651354	total: 11.1s	remaining: 13s
461:	learn: 8684.1224570	total: 11.1s	remaining: 13s
462:	learn: 8656.3838961	total: 11.2s	remaining: 13s
463:	learn: 8632.3239897	total: 11.2s	remaining: 12.9s
464:	learn: 8603.0210386	total: 11.2s	remaining: 12.9s
465:	learn: 8596.9022756	total: 11.2s	remaining: 12.9s
466:	learn: 8590.4096348	total: 11.3s	remaining: 12.8s
467:	learn: 8563.6682238	total: 11.3s	remaining: 12.8s
468:	learn: 8540.4888843	tot

610:	learn: 6064.6758190	total: 14.2s	remaining: 9.07s
611:	learn: 6048.8913034	total: 14.3s	remaining: 9.05s
612:	learn: 6038.2884490	total: 14.3s	remaining: 9.03s
613:	learn: 6016.2933646	total: 14.3s	remaining: 9s
614:	learn: 6006.8517494	total: 14.3s	remaining: 8.97s
615:	learn: 5992.1871714	total: 14.4s	remaining: 8.95s
616:	learn: 5969.6148988	total: 14.4s	remaining: 8.93s
617:	learn: 5950.1827551	total: 14.4s	remaining: 8.9s
618:	learn: 5940.2408059	total: 14.4s	remaining: 8.88s
619:	learn: 5925.2845340	total: 14.5s	remaining: 8.86s
620:	learn: 5916.5814810	total: 14.5s	remaining: 8.84s
621:	learn: 5908.0534662	total: 14.5s	remaining: 8.81s
622:	learn: 5897.6143047	total: 14.5s	remaining: 8.79s
623:	learn: 5884.9570090	total: 14.5s	remaining: 8.76s
624:	learn: 5874.8199867	total: 14.6s	remaining: 8.73s
625:	learn: 5857.9762484	total: 14.6s	remaining: 8.71s
626:	learn: 5848.1481168	total: 14.6s	remaining: 8.68s
627:	learn: 5830.8802060	total: 14.6s	remaining: 8.65s
628:	learn: 58

764:	learn: 4361.7044081	total: 18.4s	remaining: 5.64s
765:	learn: 4350.6052488	total: 18.4s	remaining: 5.62s
766:	learn: 4347.0953173	total: 18.4s	remaining: 5.6s
767:	learn: 4334.3511043	total: 18.5s	remaining: 5.58s
768:	learn: 4324.3415996	total: 18.5s	remaining: 5.55s
769:	learn: 4315.9972657	total: 18.5s	remaining: 5.53s
770:	learn: 4309.2055193	total: 18.6s	remaining: 5.51s
771:	learn: 4303.1579017	total: 18.6s	remaining: 5.49s
772:	learn: 4294.7149227	total: 18.6s	remaining: 5.46s
773:	learn: 4289.3828642	total: 18.6s	remaining: 5.44s
774:	learn: 4282.4377122	total: 18.7s	remaining: 5.42s
775:	learn: 4276.9745753	total: 18.7s	remaining: 5.4s
776:	learn: 4268.4953483	total: 18.7s	remaining: 5.37s
777:	learn: 4257.6612489	total: 18.8s	remaining: 5.35s
778:	learn: 4248.7776971	total: 18.8s	remaining: 5.33s
779:	learn: 4242.7351437	total: 18.8s	remaining: 5.31s
780:	learn: 4235.2523901	total: 18.8s	remaining: 5.28s
781:	learn: 4225.1112363	total: 18.9s	remaining: 5.26s
782:	learn: 

916:	learn: 3210.4555298	total: 22.5s	remaining: 2.04s
917:	learn: 3207.9780164	total: 22.5s	remaining: 2.01s
918:	learn: 3200.1102091	total: 22.6s	remaining: 1.99s
919:	learn: 3192.6615462	total: 22.6s	remaining: 1.97s
920:	learn: 3183.5430982	total: 22.6s	remaining: 1.94s
921:	learn: 3180.3491536	total: 22.7s	remaining: 1.92s
922:	learn: 3171.2203006	total: 22.7s	remaining: 1.9s
923:	learn: 3166.6051364	total: 22.8s	remaining: 1.87s
924:	learn: 3158.5630264	total: 22.8s	remaining: 1.85s
925:	learn: 3151.2144523	total: 22.8s	remaining: 1.82s
926:	learn: 3139.2583889	total: 22.9s	remaining: 1.8s
927:	learn: 3137.7939141	total: 22.9s	remaining: 1.78s
928:	learn: 3133.5342753	total: 22.9s	remaining: 1.75s
929:	learn: 3125.3872930	total: 23s	remaining: 1.73s
930:	learn: 3121.6270277	total: 23s	remaining: 1.71s
931:	learn: 3114.2745072	total: 23.1s	remaining: 1.68s
932:	learn: 3107.3717727	total: 23.1s	remaining: 1.66s
933:	learn: 3097.1884740	total: 23.1s	remaining: 1.63s
934:	learn: 3088

In [363]:
train,test = encoder()
er = VotingRegressor([
                      ('RandomForestRegressor', RandomForestRegressor(n_estimators=300)), 
                      ('CatBoostRegressor', CatBoostRegressor()),
                      ('Lasso', Lasso()),
                      ('CatBoostRegressor2', CatBoostRegressor()),
                     ], weights=[1,2,1,1])
er.fit(train,y_train)
print(mean_squared_error(y_test, er.predict(test), squared=False))

Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 4.27ms	remaining: 4.27s
1:	learn: 73575.3956012	total: 8.57ms	remaining: 4.27s
2:	learn: 71722.9144612	total: 13ms	remaining: 4.3s
3:	learn: 70033.8231191	total: 17.2ms	remaining: 4.28s
4:	learn: 68312.9975773	total: 22.2ms	remaining: 4.42s
5:	learn: 66800.0626965	total: 26.3ms	remaining: 4.36s
6:	learn: 65197.7255418	total: 30.4ms	remaining: 4.32s
7:	learn: 63706.4232338	total: 36.3ms	remaining: 4.5s
8:	learn: 62233.4227708	total: 40.8ms	remaining: 4.49s
9:	learn: 60819.7728303	total: 44.9ms	remaining: 4.45s
10:	learn: 59475.2948358	total: 49.1ms	remaining: 4.42s
11:	learn: 58301.3539842	total: 53.5ms	remaining: 4.41s
12:	learn: 57025.9811731	total: 57.8ms	remaining: 4.38s
13:	learn: 55853.2653197	total: 61.9ms	remaining: 4.36s
14:	learn: 54656.6746294	total: 66.2ms	remaining: 4.35s
15:	learn: 53548.0073731	total: 70.5ms	remaining: 4.33s
16:	learn: 52408.8573187	total: 74.7ms	remaining: 4.32s
17:	learn: 51297.6715207	total: 

167:	learn: 18236.4892638	total: 789ms	remaining: 3.91s
168:	learn: 18199.5215238	total: 795ms	remaining: 3.91s
169:	learn: 18182.6801213	total: 801ms	remaining: 3.91s
170:	learn: 18130.2004547	total: 807ms	remaining: 3.91s
171:	learn: 18107.6641165	total: 813ms	remaining: 3.91s
172:	learn: 18081.8818769	total: 818ms	remaining: 3.91s
173:	learn: 18035.1784031	total: 823ms	remaining: 3.9s
174:	learn: 17987.8209398	total: 827ms	remaining: 3.9s
175:	learn: 17961.2862258	total: 831ms	remaining: 3.89s
176:	learn: 17907.7687165	total: 835ms	remaining: 3.88s
177:	learn: 17869.6092229	total: 839ms	remaining: 3.88s
178:	learn: 17813.5374091	total: 845ms	remaining: 3.88s
179:	learn: 17761.9685919	total: 849ms	remaining: 3.87s
180:	learn: 17711.5766537	total: 854ms	remaining: 3.86s
181:	learn: 17689.3815877	total: 858ms	remaining: 3.86s
182:	learn: 17659.8585436	total: 862ms	remaining: 3.85s
183:	learn: 17645.8539648	total: 866ms	remaining: 3.84s
184:	learn: 17631.2646328	total: 871ms	remaining: 

342:	learn: 13009.7271926	total: 1.56s	remaining: 3s
343:	learn: 13006.8473988	total: 1.57s	remaining: 2.99s
344:	learn: 12986.3632598	total: 1.57s	remaining: 2.99s
345:	learn: 12966.3153513	total: 1.58s	remaining: 2.98s
346:	learn: 12942.0147857	total: 1.58s	remaining: 2.98s
347:	learn: 12918.5293914	total: 1.59s	remaining: 2.98s
348:	learn: 12892.2306530	total: 1.6s	remaining: 2.98s
349:	learn: 12870.2920245	total: 1.6s	remaining: 2.98s
350:	learn: 12837.1951353	total: 1.61s	remaining: 2.97s
351:	learn: 12816.4665972	total: 1.61s	remaining: 2.97s
352:	learn: 12796.5872819	total: 1.62s	remaining: 2.96s
353:	learn: 12786.0039150	total: 1.62s	remaining: 2.96s
354:	learn: 12757.6797500	total: 1.63s	remaining: 2.95s
355:	learn: 12722.5677136	total: 1.63s	remaining: 2.95s
356:	learn: 12708.0247938	total: 1.63s	remaining: 2.94s
357:	learn: 12681.7245539	total: 1.64s	remaining: 2.94s
358:	learn: 12669.6405676	total: 1.64s	remaining: 2.93s
359:	learn: 12644.2519531	total: 1.65s	remaining: 2.9

514:	learn: 10145.1967538	total: 2.34s	remaining: 2.21s
515:	learn: 10127.9456372	total: 2.35s	remaining: 2.2s
516:	learn: 10124.2336981	total: 2.35s	remaining: 2.2s
517:	learn: 10108.0595440	total: 2.36s	remaining: 2.19s
518:	learn: 10096.5632717	total: 2.36s	remaining: 2.19s
519:	learn: 10074.9803325	total: 2.37s	remaining: 2.19s
520:	learn: 10069.2108262	total: 2.38s	remaining: 2.18s
521:	learn: 10051.1007662	total: 2.38s	remaining: 2.18s
522:	learn: 10048.7969716	total: 2.38s	remaining: 2.17s
523:	learn: 10037.4340523	total: 2.39s	remaining: 2.17s
524:	learn: 10029.8315476	total: 2.39s	remaining: 2.17s
525:	learn: 10008.9021243	total: 2.4s	remaining: 2.16s
526:	learn: 9995.2591996	total: 2.4s	remaining: 2.16s
527:	learn: 9991.5020439	total: 2.41s	remaining: 2.15s
528:	learn: 9989.5751218	total: 2.41s	remaining: 2.15s
529:	learn: 9979.4950084	total: 2.42s	remaining: 2.14s
530:	learn: 9965.6907126	total: 2.42s	remaining: 2.14s
531:	learn: 9956.4093602	total: 2.42s	remaining: 2.13s
53

685:	learn: 8343.4746885	total: 3.13s	remaining: 1.43s
686:	learn: 8331.0209699	total: 3.14s	remaining: 1.43s
687:	learn: 8314.5723427	total: 3.15s	remaining: 1.43s
688:	learn: 8301.6326765	total: 3.16s	remaining: 1.43s
689:	learn: 8288.8759227	total: 3.17s	remaining: 1.42s
690:	learn: 8286.4797483	total: 3.17s	remaining: 1.42s
691:	learn: 8283.9923875	total: 3.17s	remaining: 1.41s
692:	learn: 8268.1600567	total: 3.18s	remaining: 1.41s
693:	learn: 8267.2119598	total: 3.18s	remaining: 1.4s
694:	learn: 8256.2257423	total: 3.19s	remaining: 1.4s
695:	learn: 8249.7259099	total: 3.19s	remaining: 1.39s
696:	learn: 8242.7705533	total: 3.2s	remaining: 1.39s
697:	learn: 8227.4761808	total: 3.2s	remaining: 1.38s
698:	learn: 8221.4735492	total: 3.21s	remaining: 1.38s
699:	learn: 8219.9752116	total: 3.21s	remaining: 1.38s
700:	learn: 8212.5105803	total: 3.21s	remaining: 1.37s
701:	learn: 8205.3663362	total: 3.22s	remaining: 1.36s
702:	learn: 8198.6040789	total: 3.22s	remaining: 1.36s
703:	learn: 81

...

217:	learn: 16593.0166082	total: 983ms	remaining: 3.53s
218:	learn: 16583.7557847	total: 988ms	remaining: 3.52s
219:	learn: 16552.8944294	total: 993ms	remaining: 3.52s
220:	learn: 16511.0720994	total: 999ms	remaining: 3.52s
221:	learn: 16482.6773510	total: 1s	remaining: 3.52s
222:	learn: 16455.5427377	total: 1.01s	remaining: 3.52s
223:	learn: 16414.4418901	total: 1.01s	remaining: 3.51s
224:	learn: 16401.4164098	total: 1.02s	remaining: 3.5s
225:	learn: 16366.4693791	total: 1.02s	remaining: 3.51s
226:	learn: 16337.0264555	total: 1.03s	remaining: 3.5s
227:	learn: 16292.6424399	total: 1.03s	remaining: 3.5s
228:	learn: 16281.8681598	total: 1.04s	remaining: 3.49s
229:	learn: 16253.7348871	total: 1.04s	remaining: 3.48s
230:	learn: 16245.0626352	total: 1.04s	remaining: 3.47s
231:	learn: 16206.4744806	total: 1.05s	remaining: 3.47s
232:	learn: 16154.7491503	total: 1.05s	remaining: 3.46s
233:	learn: 16120.8714155	total: 1.05s	remaining: 3.46s
234:	learn: 16080.9338124	total: 1.06s	remaining: 3.45

390:	learn: 12017.2828716	total: 1.76s	remaining: 2.75s
391:	learn: 11993.9219519	total: 1.77s	remaining: 2.75s
392:	learn: 11977.3000455	total: 1.78s	remaining: 2.75s
393:	learn: 11969.8112112	total: 1.78s	remaining: 2.74s
394:	learn: 11953.0779665	total: 1.79s	remaining: 2.74s
395:	learn: 11928.4736954	total: 1.79s	remaining: 2.73s
396:	learn: 11904.6196053	total: 1.8s	remaining: 2.73s
397:	learn: 11883.3241675	total: 1.8s	remaining: 2.73s
398:	learn: 11870.5938797	total: 1.81s	remaining: 2.72s
399:	learn: 11854.4419729	total: 1.81s	remaining: 2.72s
400:	learn: 11852.3083432	total: 1.82s	remaining: 2.71s
401:	learn: 11836.0142843	total: 1.82s	remaining: 2.71s
402:	learn: 11820.3567118	total: 1.83s	remaining: 2.71s
403:	learn: 11809.8750685	total: 1.83s	remaining: 2.7s
404:	learn: 11795.3798423	total: 1.83s	remaining: 2.69s
405:	learn: 11770.6869326	total: 1.84s	remaining: 2.69s
406:	learn: 11748.8230563	total: 1.84s	remaining: 2.69s
407:	learn: 11735.0945502	total: 1.85s	remaining: 2

560:	learn: 9572.7785800	total: 2.54s	remaining: 1.99s
561:	learn: 9557.7897504	total: 2.54s	remaining: 1.98s
562:	learn: 9547.4482241	total: 2.55s	remaining: 1.98s
563:	learn: 9541.3281935	total: 2.55s	remaining: 1.97s
564:	learn: 9525.4040966	total: 2.56s	remaining: 1.97s
565:	learn: 9515.5172046	total: 2.56s	remaining: 1.97s
566:	learn: 9503.0433265	total: 2.57s	remaining: 1.96s
567:	learn: 9494.6687943	total: 2.57s	remaining: 1.96s
568:	learn: 9481.3174836	total: 2.58s	remaining: 1.96s
569:	learn: 9463.7910987	total: 2.59s	remaining: 1.95s
570:	learn: 9451.3795127	total: 2.59s	remaining: 1.95s
571:	learn: 9434.9569502	total: 2.6s	remaining: 1.94s
572:	learn: 9423.3549978	total: 2.6s	remaining: 1.94s
573:	learn: 9405.1255754	total: 2.61s	remaining: 1.93s
574:	learn: 9386.8695655	total: 2.61s	remaining: 1.93s
575:	learn: 9374.1838713	total: 2.62s	remaining: 1.92s
576:	learn: 9361.2474634	total: 2.62s	remaining: 1.92s
577:	learn: 9343.7427041	total: 2.62s	remaining: 1.92s
578:	learn: 

729:	learn: 7933.2251831	total: 3.31s	remaining: 1.22s
730:	learn: 7923.9440371	total: 3.32s	remaining: 1.22s
731:	learn: 7906.4985764	total: 3.32s	remaining: 1.22s
732:	learn: 7897.8615232	total: 3.33s	remaining: 1.21s
733:	learn: 7896.8964321	total: 3.33s	remaining: 1.21s
734:	learn: 7884.5527779	total: 3.34s	remaining: 1.2s
735:	learn: 7873.6435940	total: 3.34s	remaining: 1.2s
736:	learn: 7865.1990261	total: 3.35s	remaining: 1.19s
737:	learn: 7850.8117468	total: 3.35s	remaining: 1.19s
738:	learn: 7838.7050952	total: 3.35s	remaining: 1.19s
739:	learn: 7833.4283643	total: 3.36s	remaining: 1.18s
740:	learn: 7827.3992781	total: 3.37s	remaining: 1.18s
741:	learn: 7821.7490544	total: 3.37s	remaining: 1.17s
742:	learn: 7820.6485709	total: 3.38s	remaining: 1.17s
743:	learn: 7815.9668755	total: 3.38s	remaining: 1.16s
744:	learn: 7799.6624680	total: 3.38s	remaining: 1.16s
745:	learn: 7791.4214607	total: 3.39s	remaining: 1.15s
746:	learn: 7790.6631426	total: 3.39s	remaining: 1.15s
747:	learn: 

893:	learn: 6682.7112978	total: 4.3s	remaining: 510ms
894:	learn: 6682.0076149	total: 4.31s	remaining: 506ms
895:	learn: 6671.8570567	total: 4.31s	remaining: 501ms
896:	learn: 6671.2315801	total: 4.32s	remaining: 496ms
897:	learn: 6662.9641221	total: 4.33s	remaining: 492ms
898:	learn: 6655.8693721	total: 4.33s	remaining: 487ms
899:	learn: 6649.6832759	total: 4.35s	remaining: 483ms
900:	learn: 6640.9003457	total: 4.35s	remaining: 478ms
901:	learn: 6625.2508103	total: 4.36s	remaining: 474ms
902:	learn: 6622.7719699	total: 4.37s	remaining: 469ms
903:	learn: 6622.1144047	total: 4.38s	remaining: 465ms
904:	learn: 6616.7891799	total: 4.39s	remaining: 461ms
905:	learn: 6610.8228211	total: 4.39s	remaining: 456ms
906:	learn: 6599.8996485	total: 4.4s	remaining: 451ms
907:	learn: 6588.5330009	total: 4.4s	remaining: 446ms
908:	learn: 6582.6334160	total: 4.41s	remaining: 441ms
909:	learn: 6574.5204105	total: 4.41s	remaining: 436ms
910:	learn: 6571.7881222	total: 4.42s	remaining: 432ms
911:	learn: 6

In [368]:
er.score(train,y_train)

0.987979062741465

In [ ]:
train,test = encoder()
sr = StackingRegressor(estimators=[
                      ('RandomForestRegressor', RandomForestRegressor(n_estimators=300)), 
                      ('CatBoostRegressor', CatBoostRegressor()),
                      ('Lasso', Lasso())
                      #('LGBMRegressor', LGBMRegressor())
                     ], final_estimator=Lasso())
sr.fit(train,y_train)
print(mean_squared_error(y_test, sr.predict(test), squared=False))

Learning rate set to 0.040124
0:	learn: 75303.7426967	total: 5.46ms	remaining: 5.46s
1:	learn: 73575.3956012	total: 11.3ms	remaining: 5.66s
2:	learn: 71722.9144612	total: 15.7ms	remaining: 5.2s
3:	learn: 70033.8231191	total: 19.8ms	remaining: 4.94s
4:	learn: 68312.9975773	total: 24.2ms	remaining: 4.81s
5:	learn: 66800.0626965	total: 28.4ms	remaining: 4.7s
6:	learn: 65197.7255418	total: 32.5ms	remaining: 4.61s
7:	learn: 63706.4232338	total: 36.8ms	remaining: 4.56s
8:	learn: 62233.4227708	total: 41.2ms	remaining: 4.53s
9:	learn: 60819.7728303	total: 45.4ms	remaining: 4.49s
10:	learn: 59475.2948358	total: 49.7ms	remaining: 4.47s
11:	learn: 58301.3539842	total: 54ms	remaining: 4.45s
12:	learn: 57025.9811731	total: 58.2ms	remaining: 4.42s
13:	learn: 55853.2653197	total: 62.3ms	remaining: 4.39s
14:	learn: 54656.6746294	total: 66.6ms	remaining: 4.37s
15:	learn: 53548.0073731	total: 70.7ms	remaining: 4.35s
16:	learn: 52408.8573187	total: 75.2ms	remaining: 4.35s
17:	learn: 51297.6715207	total: 

153:	learn: 18970.1533044	total: 991ms	remaining: 5.45s
154:	learn: 18906.9813986	total: 999ms	remaining: 5.44s
155:	learn: 18834.2813827	total: 1.01s	remaining: 5.45s
156:	learn: 18788.7469764	total: 1.01s	remaining: 5.45s
157:	learn: 18736.2958776	total: 1.02s	remaining: 5.44s
158:	learn: 18695.6087362	total: 1.03s	remaining: 5.44s
159:	learn: 18647.6612140	total: 1.03s	remaining: 5.43s
160:	learn: 18590.6168358	total: 1.04s	remaining: 5.43s
161:	learn: 18524.0864836	total: 1.05s	remaining: 5.42s
162:	learn: 18467.3296157	total: 1.05s	remaining: 5.41s
163:	learn: 18396.7995948	total: 1.06s	remaining: 5.41s
164:	learn: 18355.8102288	total: 1.07s	remaining: 5.42s
165:	learn: 18335.2389905	total: 1.08s	remaining: 5.42s
166:	learn: 18288.6781242	total: 1.09s	remaining: 5.42s
167:	learn: 18236.4892638	total: 1.09s	remaining: 5.42s
168:	learn: 18199.5215238	total: 1.1s	remaining: 5.43s
169:	learn: 18182.6801213	total: 1.11s	remaining: 5.41s
170:	learn: 18130.2004547	total: 1.11s	remaining:

305:	learn: 13890.8918912	total: 2.19s	remaining: 4.98s
306:	learn: 13859.6282614	total: 2.2s	remaining: 4.97s
307:	learn: 13837.1781098	total: 2.21s	remaining: 4.97s
308:	learn: 13811.5105708	total: 2.22s	remaining: 4.97s
309:	learn: 13776.6027791	total: 2.23s	remaining: 4.97s
310:	learn: 13747.9562354	total: 2.24s	remaining: 4.96s
311:	learn: 13723.3290943	total: 2.25s	remaining: 4.96s
312:	learn: 13691.2233559	total: 2.25s	remaining: 4.95s
313:	learn: 13655.2220110	total: 2.26s	remaining: 4.94s
314:	learn: 13619.6506908	total: 2.27s	remaining: 4.93s
315:	learn: 13591.4332612	total: 2.27s	remaining: 4.92s
316:	learn: 13557.7732694	total: 2.28s	remaining: 4.91s
317:	learn: 13536.4416987	total: 2.29s	remaining: 4.91s
318:	learn: 13513.7901844	total: 2.29s	remaining: 4.9s
319:	learn: 13509.5653817	total: 2.3s	remaining: 4.89s
320:	learn: 13482.0629166	total: 2.31s	remaining: 4.88s
321:	learn: 13462.2641475	total: 2.32s	remaining: 4.89s
322:	learn: 13442.4558410	total: 2.33s	remaining: 4

486:	learn: 10506.6433463	total: 3.19s	remaining: 3.36s
487:	learn: 10484.2717572	total: 3.19s	remaining: 3.35s
488:	learn: 10465.2294654	total: 3.2s	remaining: 3.34s
489:	learn: 10456.4707949	total: 3.2s	remaining: 3.33s
490:	learn: 10434.5298913	total: 3.21s	remaining: 3.33s
491:	learn: 10415.4949262	total: 3.21s	remaining: 3.32s
492:	learn: 10399.1718296	total: 3.22s	remaining: 3.31s
493:	learn: 10381.0123115	total: 3.23s	remaining: 3.3s
494:	learn: 10378.7735443	total: 3.23s	remaining: 3.3s
495:	learn: 10363.8012184	total: 3.23s	remaining: 3.29s
496:	learn: 10346.0385310	total: 3.24s	remaining: 3.28s
497:	learn: 10330.2565401	total: 3.25s	remaining: 3.27s
498:	learn: 10320.1869572	total: 3.25s	remaining: 3.26s
499:	learn: 10314.3938247	total: 3.25s	remaining: 3.25s
500:	learn: 10305.2624371	total: 3.26s	remaining: 3.25s
501:	learn: 10292.5867978	total: 3.27s	remaining: 3.24s
502:	learn: 10273.9291716	total: 3.28s	remaining: 3.24s
503:	learn: 10248.2022145	total: 3.29s	remaining: 3.

667:	learn: 8524.0472050	total: 4.58s	remaining: 2.27s
668:	learn: 8507.4768905	total: 4.58s	remaining: 2.27s
669:	learn: 8498.6391181	total: 4.59s	remaining: 2.26s
670:	learn: 8491.4867861	total: 4.59s	remaining: 2.25s
671:	learn: 8481.7454797	total: 4.6s	remaining: 2.24s
672:	learn: 8471.2407901	total: 4.6s	remaining: 2.24s
673:	learn: 8466.5793539	total: 4.61s	remaining: 2.23s
674:	learn: 8457.1890479	total: 4.62s	remaining: 2.22s
675:	learn: 8453.3252804	total: 4.63s	remaining: 2.22s
676:	learn: 8441.3827722	total: 4.64s	remaining: 2.21s
677:	learn: 8432.6723113	total: 4.64s	remaining: 2.21s
678:	learn: 8413.1784692	total: 4.65s	remaining: 2.2s
679:	learn: 8405.0547936	total: 4.66s	remaining: 2.19s
680:	learn: 8401.6567782	total: 4.66s	remaining: 2.18s
681:	learn: 8385.2898327	total: 4.67s	remaining: 2.18s
682:	learn: 8372.7835640	total: 4.67s	remaining: 2.17s
683:	learn: 8363.7749390	total: 4.68s	remaining: 2.16s
684:	learn: 8360.0675651	total: 4.69s	remaining: 2.15s
685:	learn: 8

...

160:	learn: 18766.5422295	total: 999ms	remaining: 5.21s
161:	learn: 18708.1449294	total: 1s	remaining: 5.19s
162:	learn: 18656.5537597	total: 1.01s	remaining: 5.18s
163:	learn: 18584.4444068	total: 1.02s	remaining: 5.21s
164:	learn: 18524.8544261	total: 1.03s	remaining: 5.21s
165:	learn: 18474.0347951	total: 1.03s	remaining: 5.2s
166:	learn: 18438.1263195	total: 1.04s	remaining: 5.18s
167:	learn: 18391.0699386	total: 1.04s	remaining: 5.16s
168:	learn: 18344.3278188	total: 1.05s	remaining: 5.15s
169:	learn: 18281.2024186	total: 1.05s	remaining: 5.14s
170:	learn: 18255.3629515	total: 1.06s	remaining: 5.13s
171:	learn: 18190.2766969	total: 1.06s	remaining: 5.12s
172:	learn: 18180.3786018	total: 1.07s	remaining: 5.11s
173:	learn: 18146.3913171	total: 1.07s	remaining: 5.1s
174:	learn: 18089.8587769	total: 1.08s	remaining: 5.09s
175:	learn: 18067.9152504	total: 1.08s	remaining: 5.07s
176:	learn: 18041.4256601	total: 1.09s	remaining: 5.05s
177:	learn: 18003.0098973	total: 1.09s	remaining: 5.0

330:	learn: 13575.9478218	total: 1.99s	remaining: 4.02s
331:	learn: 13571.8101995	total: 2s	remaining: 4.01s
332:	learn: 13555.8447296	total: 2.01s	remaining: 4.02s
333:	learn: 13537.3138425	total: 2.01s	remaining: 4.01s
334:	learn: 13513.2549403	total: 2.02s	remaining: 4.01s
335:	learn: 13482.7149886	total: 2.03s	remaining: 4.01s
336:	learn: 13471.3100029	total: 2.04s	remaining: 4.01s
337:	learn: 13442.4905738	total: 2.05s	remaining: 4.01s
338:	learn: 13421.2417366	total: 2.06s	remaining: 4.01s
339:	learn: 13400.4537385	total: 2.06s	remaining: 4.01s
340:	learn: 13396.5806774	total: 2.07s	remaining: 4s
341:	learn: 13373.7645015	total: 2.08s	remaining: 3.99s
342:	learn: 13359.4791829	total: 2.08s	remaining: 3.98s
343:	learn: 13326.1113925	total: 2.08s	remaining: 3.98s
344:	learn: 13313.5107172	total: 2.09s	remaining: 3.97s
345:	learn: 13287.6817252	total: 2.1s	remaining: 3.96s
346:	learn: 13253.1366002	total: 2.1s	remaining: 3.96s
347:	learn: 13222.1072802	total: 2.11s	remaining: 3.95s


493:	learn: 10465.5554533	total: 2.98s	remaining: 3.05s
494:	learn: 10441.7101914	total: 2.98s	remaining: 3.04s
495:	learn: 10418.3262838	total: 3s	remaining: 3.05s
496:	learn: 10395.8006680	total: 3.02s	remaining: 3.05s
497:	learn: 10384.0150964	total: 3.02s	remaining: 3.04s
498:	learn: 10366.1269716	total: 3.03s	remaining: 3.04s
499:	learn: 10339.2007852	total: 3.03s	remaining: 3.03s
500:	learn: 10317.3985856	total: 3.04s	remaining: 3.03s
501:	learn: 10311.7090486	total: 3.05s	remaining: 3.02s
502:	learn: 10296.5711201	total: 3.05s	remaining: 3.02s
503:	learn: 10295.0057766	total: 3.06s	remaining: 3.01s
504:	learn: 10269.0361889	total: 3.07s	remaining: 3s
505:	learn: 10255.1099505	total: 3.07s	remaining: 3s
506:	learn: 10248.1045043	total: 3.08s	remaining: 3s
507:	learn: 10219.6572786	total: 3.09s	remaining: 2.99s
508:	learn: 10206.6392118	total: 3.09s	remaining: 2.98s
509:	learn: 10190.7252851	total: 3.1s	remaining: 2.98s
510:	learn: 10189.3918652	total: 3.1s	remaining: 2.97s
511:	l

660:	learn: 8196.3730553	total: 3.96s	remaining: 2.03s
661:	learn: 8184.0521592	total: 3.97s	remaining: 2.03s
662:	learn: 8180.2406627	total: 3.98s	remaining: 2.02s
663:	learn: 8166.0669414	total: 3.98s	remaining: 2.02s
664:	learn: 8164.9337744	total: 3.99s	remaining: 2.01s
665:	learn: 8153.2614971	total: 4s	remaining: 2s
666:	learn: 8148.2435006	total: 4s	remaining: 2s
667:	learn: 8138.2245226	total: 4s	remaining: 1.99s
668:	learn: 8125.4337964	total: 4.01s	remaining: 1.98s
669:	learn: 8106.1164991	total: 4.01s	remaining: 1.98s
670:	learn: 8102.1377359	total: 4.02s	remaining: 1.97s
671:	learn: 8090.7344157	total: 4.02s	remaining: 1.96s
672:	learn: 8070.1353762	total: 4.03s	remaining: 1.96s
673:	learn: 8057.1169730	total: 4.03s	remaining: 1.95s
674:	learn: 8055.9160614	total: 4.04s	remaining: 1.94s
675:	learn: 8036.5551969	total: 4.04s	remaining: 1.94s
676:	learn: 8018.2184316	total: 4.05s	remaining: 1.93s
677:	learn: 8008.8400647	total: 4.05s	remaining: 1.92s
678:	learn: 8001.3279639	

821:	learn: 6609.9142127	total: 4.95s	remaining: 1.07s
822:	learn: 6604.3328742	total: 4.95s	remaining: 1.06s
823:	learn: 6594.4351878	total: 4.96s	remaining: 1.06s
824:	learn: 6582.5779728	total: 4.97s	remaining: 1.05s
825:	learn: 6574.0371388	total: 4.97s	remaining: 1.05s
826:	learn: 6564.0864671	total: 4.98s	remaining: 1.04s
827:	learn: 6555.0774310	total: 4.98s	remaining: 1.03s
828:	learn: 6554.4221686	total: 4.99s	remaining: 1.03s
829:	learn: 6542.2152736	total: 5s	remaining: 1.02s
830:	learn: 6533.4668617	total: 5s	remaining: 1.02s
831:	learn: 6520.1433926	total: 5.01s	remaining: 1.01s
832:	learn: 6506.6125385	total: 5.01s	remaining: 1s
833:	learn: 6503.5281210	total: 5.02s	remaining: 998ms
834:	learn: 6499.5724840	total: 5.02s	remaining: 992ms
835:	learn: 6485.7818560	total: 5.03s	remaining: 986ms
836:	learn: 6473.4690185	total: 5.03s	remaining: 980ms
837:	learn: 6461.2972805	total: 5.04s	remaining: 973ms
838:	learn: 6457.7735956	total: 5.04s	remaining: 967ms
839:	learn: 6446.64

# Best model: StackingRegressor - 25467.162313549037
